# *FEniCS tutorial:* Diffusion of a Gaussian hill

In this demo, we solve the two-dimensional diffusion equation with Dirichlet
bounadary conditions $u_D$ and source term $f = 0$. The initial condition,
$u_0$, we difine as a Gaussian hill.

$$
\begin{align}
  u'   &= \nabla^2 u + f \quad\text{in the unit square} \\
  u    &= u_D  \hphantom{u+f}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{u+f}\quad\;\text{at $t = 0$} \\
\end{align}
$$
with
$$
\begin{align}
  u_D  &= 0 \\
  u_0  &= \exp\bigl(-a(x^2 + y^2)\bigr) \\
  f    &= 0
\end{align}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time as tm

# import os
# import re
# from color import *
# from IPython.display import Image
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# from mplotg import *
# import matplotlib.pyplot as plt

def multiplot_from_generator(g, num_columns, figsize_for_one_row=None):
    # default to 15-inch rows, with square subplots
    if figsize_for_one_row is None:
        figsize_for_one_row = (15, 15/num_columns)
    try:
        while True:
            # call plt.figure once per row
            plt.figure(figsize=figsize_for_one_row)
            for col in range(num_columns):
                ax = plt.subplot(1, num_columns, col+1)
                next(g)
    except StopIteration:
        pass

In [ ]:
from fenics import *

In [ ]:
## set up the problem and define main simulation function evolve()

# set parameters
T = 2.0            # total simulation time
num_steps = 50     # number of time steps
dt = T / num_steps # size of time step

# create mesh and define function space
nx = ny = 30
mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# define boundary condition
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, Constant(0), boundary)

# define initial value
u_0 = Expression('exp(-a*pow(x[0], 2) - a*pow(x[1], 2))',
                 degree=2, a=5)
u_n = interpolate(u_0, V)

# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# define time-evolution function
def evolve():
    # initialization
    nip = 10 # number of intervals between plots
    u = Function(V)
    t = 0

    # time-stepping
    for n in range(num_steps):

        # update current time
        t += dt

        # compute solution
        solve(a == L, u, bc)

        # plot solution at nip-step intervals
        if n % nip == 0:
            yield plot(u)

        # update previous solution
        u_n.assign(u)

In [ ]:
# multiplot_from_generator(evolve(), 5, (15, 3))
multiplot_from_generator(evolve(), 5)

In [ ]:
mesh

In [ ]:
# define initial condition
#u_n = interpolate(u_0, V)

# time-stepping
u = Function(V)
t = 0
for n in range(num_steps):

    # compute solution
    solve(a == L, u, bc)
    
    # plot solution and save to file
    plot(u)
    plt.savefig('heat_2D_sqr/' + 'u_plot_' + str(n) + '.png')

    # update current time and solution
    t += dt
    u_n.assign(u)

In [ ]:
xvals = mesh.coordinates()[:,0]
yvals = mesh.coordinates()[:,1]
uvals = u.compute_vertex_values()

In [ ]:
plt.pcolormesh(xvals, yvals, uvals, cmap=plt.cm.viridis)

In [ ]:
plt.pcolormesh?